In [25]:
import math
import pandas
import json
from shapely.geometry import shape, Point, Polygon
from shapely.ops import cascaded_union

In [26]:
#a place to save the permit values and number to a BG
BG_permits = pandas.read_csv('BG_permits.csv')

In [27]:
#load in all the permits 
permits = pandas.read_csv('Building_Permits.csv')

In [28]:
#load in the BG shapes
with open('cityBG_simp20_trial.geojson') as f:
    BGjs = json.load(f)

In [29]:
#find which BG each permit is in, and (if the value of the permit is > 0) add it to the # and value of the BG
count = 0
for ind, permit in permits.iterrows():
    #print(canopy)
    point = Point(permit['Longitude'], permit['Latitude'])
    for feature in BGjs['features']:
        #print(feature)
        polygon = shape(feature['geometry'])
        #print(polygon.contains(point))
        if polygon.contains(point):
            count = count + 1
            found_polygon = feature['properties']
            BG_id = found_polygon['gis_id']
            #print(BG_id)
            
            #look up value of this permit
            value = permit['Value'][1:-3]
            value = value.replace(',','')
            value = float(value)
            
            #if value > 0, add a permit to the total number, and add its value
            if value > 0:
                current_num = BG_permits.loc[BG_permits['gis_id'] == int(BG_id), 'num']
                index = BG_permits.loc[BG_permits['gis_id'] == int(BG_id)].index
                new_num = current_num + 1
                
                current_value = BG_permits.loc[BG_permits['gis_id'] == int(BG_id), 'total_value']
                new_value = current_value + value
            
                BG_permits.set_value(index, 'num', new_num)
                BG_permits.set_value(index, 'total_value', new_value)
            
            break

In [30]:
BG_permits.head()

,gis_id,num,total_value
0,530330001001,90,13796750.0
1,530330001002,25,22061612.0
2,530330001003,39,23477745.0
3,530330001004,10,50048662.0
4,530330001005,42,28313802.0


In [31]:
#save to an appropriate file
BG_permits.to_csv('BG_permit_values.csv')